<a href="https://colab.research.google.com/github/gmacauda/ara_coord/blob/main/folium_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from urllib.request import urlopen
import json
import folium

In [20]:
plz_data = "https://raw.githubusercontent.com/openZH/covid_19/master/fallzahlen_plz/PLZ_gen_epsg4326_F_KTZH_2020.json"

response = urlopen(plz_data)
geo_json = json.loads(response.read())

In [21]:
geo_json['features'][0]['properties']['PLZ']

5462.0

In [8]:
import pandas as pd

In [9]:
vacc_data = pd.read_csv("https://raw.githubusercontent.com/openZH/covid_19_vaccination_campaign_ZH/master/COVID19_Impfungen_pro_Woche_PLZ_v2.csv")
vacc_data['ratio'] = vacc_data['nCumul'] / vacc_data['population']
vacc_data['PLZ'] = pd.to_numeric(vacc_data['plz'], errors = 'coerce')

In [10]:
no_plz =  ["andere Kantone","Nachbarkantone","uebrige PLZ","unbekannt","ZH aber ausserkantonal geimpft"]

In [11]:
vacc_data = vacc_data.query('plz not in ["andere Kantone","Nachbarkantone","uebrige PLZ","unbekannt","ZH aber ausserkantonal geimpft"]')

In [12]:
booster_data = vacc_data[vacc_data["vacc_status"]  == 'booster']

In [13]:
booster_max_data = booster_data[booster_data['week_from'] == max(booster_data['week_from'])]

In [14]:
booster_max_data

,week_from,week_until,calendar_week,plz,vacc_status,nCumul,population,ratio,PLZ
50095,2022-04-25,2022-05-01,17,8001,booster,3391,5473.0,0.619587,8001.0
50098,2022-04-25,2022-05-01,17,8002,booster,5320,9620.0,0.553015,8002.0
50101,2022-04-25,2022-05-01,17,8003,booster,10250,20655.0,0.496248,8003.0
50104,2022-04-25,2022-05-01,17,8004,booster,14069,27183.0,0.517566,8004.0
50107,2022-04-25,2022-05-01,17,8005,booster,8809,15505.0,0.568139,8005.0
...,...,...,...,...,...,...,...,...,...
50824,2022-04-25,2022-05-01,17,8951,booster,906,3068.0,0.295306,8951.0
50827,2022-04-25,2022-05-01,17,8952,booster,7475,18758.0,0.398497,8952.0
50830,2022-04-25,2022-05-01,17,8953,booster,9575,27635.0,0.346481,8953.0
50833,2022-04-25,2022-05-01,17,8954,booster,1720,3970.0,0.433249,8954.0


In [24]:
m = folium.Map(location=[46, 8.5], zoom_start=3)

folium.Choropleth(
    geo_data=geo_json,
    name="choropleth",
    data=booster_max_data,
    columns=["PLZ", "ratio"],
    key_on="feature.properties.PLZ",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Booster Rate",
).add_to(m)

folium.LayerControl().add_to(m)

In [25]:
m